## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

5. Add your OpenAI key.  OpenAI is only needed for example purposes.  If you have your own model, you can substitute those in it's place.

In [ ]:
%pip install okareo

In [ ]:
%pip install openai

In [3]:
# Setup the environment for your notebook
import os

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [4]:
# Simple adhoc classifier using OpenAI'a GPT 3.5 Turbo model
import os
from openai import OpenAI
import json

client = OpenAI(api_key=OPENAI_API_KEY)

def get_turbo_classification(messages, model="gpt-3.5-turbo", 
  temperature=0, max_tokens=500):
  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, 
    max_tokens=max_tokens,
  )
  return response

USER_PROMPT_TEMPLATE = "{input}"

SUMMARIZATION_CONTEXT_TEMPLATE = """
You will be provided with text.
Summarize the text in 1-3 simple sentences.
If the text's title and the author's name are available, refer to both in the summary.
Your goal is to make the summary clear to a middle school student.
"""


In [5]:
# Text for the summarization model to process
text_1 = """
I Wandered Lonely as a Cloud

I wandered lonely as a cloud
That floats on high o'er vales and hills,
When all at once I saw a crowd,
A host, of golden daffodils;
Beside the lake, beneath the trees,
Fluttering and dancing in the breeze.

Continuous as the stars that shine
And twinkle on the milky way,
They stretched in never-ending line
Along the margin of a bay:
Ten thousand saw I at a glance,
Tossing their heads in sprightly dance.

The waves beside them danced; but they
Out-did the sparkling waves in glee:
A poet could not but be gay,
In such a jocund company:
I gazed-and gazed-but little thought
What wealth the show to me had brought:

For oft, when on my couch I lie
In vacant or in pensive mood,
They flash upon that inward eye
Which is the bliss of solitude;
And then my heart with pleasure fills,
And dances with the daffodils.

-- by William Wordsworth
"""

text_2 = """
All Things Are Current Found

ALL things are current found
On earthly ground,
Spirits and elements
Have their descents.

Night and day, year on year,
High and low, far and near,
These are our own aspects,
These are our own regrets.

Ye gods of the shore,
Who abide evermore,
I see you far headland,
Stretching on either hand;

I hear the sweet evening sounds
From your undecaying grounds;
Cheat me no more with time,
Take me to your clime.

-- by Henry D. Thoreau
"""

text_3 = """
Hope Is the Thing with Feathers

"Hope" is the thing with feathers-
That perches in the soul-
And sings the tune without the words-
And never stops-at all-

And sweetest-in the Gale-is heard-
And sore must be the storm-
That could abash the little Bird
That kept so many warm-

I've heard it in the chillest land-
And on the strangest Sea-
Yet, never, in Extremity,
It asked a crumb-of Me.

-- by Emily Dickinson
"""

In [6]:
# Example calling to the classifier to play with how it responds to different inputs

messages =  [  
    {'role':'system', 'content': SUMMARIZATION_CONTEXT_TEMPLATE},    
    {'role':'user', 'content': text_3}
] 

response = get_turbo_classification(messages)
print(response)

ChatCompletion(id='chatcmpl-8fcxfzQsL3ATaWgfhotNpDV9j89Op', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The poem "Hope Is the Thing with Feathers" by Emily Dickinson describes hope as a bird that lives in our souls and sings even when things are tough. It is always there for us, no matter what.', role='assistant', function_call=None, tool_calls=None))], created=1704931907, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=43, prompt_tokens=176, total_tokens=219))


In [7]:
# Create a scenario to evaluate the model with
import os
import random
import string
import tempfile
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData, ScenarioType

okareo = Okareo(OKAREO_API_KEY)

# Step A - Create seed data and save in templ jsonl file
poems = [
    {
        "input":text_1,
        "result":''.join(random.choices(string.ascii_letters, k=32))
    },
    {
        "input":text_2,
        "result":''.join(random.choices(string.ascii_letters, k=32))
    },
    {
        "input":text_3, 
        "result":''.join(random.choices(string.ascii_letters, k=32))
    },
]

temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "seed_data_summarization.jsonl")

# Write to a .jsonl file
with open(file_path, "w+") as file:
    for row in poems:
        file.write(json.dumps(row) + '\n')
    

# Step B - Create scenario set from seed data file
random_string = ''.join(random.choices(string.ascii_letters, k=5))
okareo = Okareo(OKAREO_API_KEY)

scenario = okareo.upload_scenario_set(file_path=file_path, scenario_name=f"Summarization - {random_string}")

# make sure to clean up tmp file
os.remove(file_path)

SCENARIO_ID = scenario.scenario_id
PROJECT_ID = scenario.project_id
print(scenario)
print(f"https://app.okareo.com/project/{PROJECT_ID}/scenario/{SCENARIO_ID}")

ScenarioSetResponse(scenario_id='d0004aba-9429-45dd-92a5-ade68338a3bd', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 11, 0, 11, 48, 745419), type='SEED', tags=None, name='Summarization - QSwHn', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})
https://app.okareo.com/project/f7bfcff0-2b8d-4fec-ab20-51669cd3e732/scenario/d0004aba-9429-45dd-92a5-ade68338a3bd


In [8]:
# Evaluate the scenario and model combination and then get a link to the results on Okareo
import os
import random
import string
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData
from okareo.model_under_test import OpenAIModel
from okareo_api_client.models.test_run_type import TestRunType

okareo = Okareo(OKAREO_API_KEY)
random_string = ''.join(random.choices(string.ascii_letters, k=5))
mut_name = f"OpenAI Summarization Model - {random_string}"
eval_name = f"Summarization Run - {random_string}"

model_under_test = okareo.register_model(
    name=mut_name,
    model=OpenAIModel(
        model_id="gpt-3.5-turbo",
        temperature=0,
        system_prompt_template=SUMMARIZATION_CONTEXT_TEMPLATE,
        user_prompt_template=USER_PROMPT_TEMPLATE,
    ),
)

evaluation = model_under_test.run_test(
    name=eval_name,
    scenario=scenario,
    api_key=OPENAI_API_KEY,
    test_run_type=TestRunType.NL_GENERATION,
    calculate_metrics=True,
)


EVAL_ID = evaluation.id
PROJECT_ID = evaluation.project_id
print(f"https://app.okareo.com/project/{PROJECT_ID}/eval/{EVAL_ID}")

https://app.okareo.com/project/f7bfcff0-2b8d-4fec-ab20-51669cd3e732/eval/e63eea12-ec76-4c4f-9e0d-1816379f4c39
